In [2]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [408]:
I = json.dumps({'status': 'idle', 'origin': None, 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 1, 'name': 'driver_1', 'p2': None, 'p1': None, 'ctime': '2016-09-30 23:36:03.753770', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.7778, -73.9448]})
I = json.loads(I)
ctime = I['ctime']
driver = I ['id']
status = I ['status']
location = I['location']
name = I ["name"]

try:
    tmp = datetime.strptime("{}".format(ctime), '%Y-%m-%d %H:%M:%S.%f')
    ctime = tmp.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
except:
    pass


In [409]:
geo_query = { "from" : 0, "size" : 1,
                      "query": {"filtered": { "filter": { "geo_distance": \
                          { "location": location,  "distance_type": "plane", "distance": "100km" }}, "query": { "query_string": { "query": "wait" }}}}}

res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])

if len(res['hits']['hits'])>0: 
    passenger = res['hits']['hits'][0]["_source"]
    doc = json.dumps({"status": "pickup", "driver": driver})
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='passenger', doc_type='rolling', id=passenger['id'], body=q, ignore=[400,404])
    doc = {"status": "pickup", "ctime": ctime, \
           "location": location, 'name': name, \
           'destination':  passenger['location'], \
           'destinationid': passenger['id']}
    if not p1:
        doc['p1'] = passenger['id']
    elif not p2:
        doc['p2'] = passenger['id']
    else:
        pass # It's full!
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    doc = {"status": "idle", "ctime": ctime, "location": location, 'name': name}
    doc = json.dumps(doc)
    q = '{{"doc": {}, "doc_as_upsert": "true"}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                    body=q)
    print doc
else:
    doc = {"ctime": ctime, "location": location}
    doc = json.dumps(doc)
    q = '{{"doc": {}}}'.format(doc)
    res = es.update(index='driver', doc_type='rolling', id=driver, \
                        body=q)
    print(doc)

{"ctime": "2016-09-30T23:36:03.753770Z", "location": [40.7778, -73.9448]}


In [37]:
last_uid = 1
q = es.get(index='passenger', doc_type='rolling', id=last_uid, ignore=[404, 400])


In [59]:
q = { 'size': 1,
    'query' : {
            'term' : { 'status' : 'arrived' }
    },
    "sort": [
    {
      "ctime": {
        "order": "asc"
      }}]
    }
res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])


if len(res['hits']['hits']) > 0:
    res = res['hits']['hits'][0]['_source']
    msg = ["Driver {} just arrived to {}".format(\
            res['id'], res['destinationid']), res['destination']]

In [56]:
import csv
import random
import time
import json

from kafka import KafkaClient, KeyedProducer, SimpleConsumer
from datetime import datetime, timedelta
from decimal import *

city = random.choice(['CHI','SFO','NYC'])


In [45]:
boundaries_file = "../../kafka/boundaries.csv"
tourist_attractions = "../../kafka/destinations.csv"



In [127]:
# This is the script to populate driver's Data
# {driver_id, time, curr_lat, curr_long, dest, load}
total_drivers = 5 


import time
import json
import random 
import csv
import random

from decimal import *
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
from elasticsearch import Elasticsearch


'''
        Set the geographical boundaries and other variables
'''
boundaries_file = "../../kafka/boundaries.csv"
city = random.choice(['CHI','NYC'])

getcontext().prec=6
step_to_dest = random.randrange(1,2)

cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', \
                    'ip-172-31-0-105', 'ip-172-31-0-106']

brokers = ','.join(['{}:9092'.format(i) for i in cluster])
#brokers = 'localhost:9092'
kafka = KafkaClient(brokers)
producer = KeyedProducer(kafka)
es = Elasticsearch(cluster, \
                   port=9200)



def loadBoundaries(boundaries_file):

    boundaries = {}
    with open(boundaries_file, 'r') as f:
        reader = csv.reader(f, delimiter = ',')
        for row in reader:
            city = row[0]
            fence = row[1:] 
            boundaries.update({city:fence})
    f.close()
    return boundaries

def simulateTrip(i, city):
    bnd = bound[city]
    q = es.get(index='driver', doc_type='rolling', id=i, ignore=[404, 400])
    if q['found']:
        if (q['_source']['status'] in ['ontrip', 'pickup']): 
            d = q['_source']['destination']
            c = q['_source']['location']
            la = Decimal(c[0]) + (((Decimal(d[0]) - Decimal(c[0]))/step_to_dest))
            lo = Decimal(c[1]) + (((Decimal(d[1]) - Decimal(c[1]))/step_to_dest))
            return [float(str(la)), float(str(lo))]
        else:
            return [round(random.uniform(float(bnd[0]), float(bnd[2])),4),\
                round(random.uniform(float(bnd[1]), float(bnd[3])),4)]

    else:
            return [round(random.uniform(float(bnd[0]), float(bnd[2])),4),\
                round(random.uniform(float(bnd[1]), float(bnd[3])),4)]
    
def generateDriver(city):
    d_id = random.randint(1, total_drivers)
    driver_mapping ={ 
            'city': city,
            'name': 'driver_{}'.format(d_id),
            'id': d_id,
            'status': 'idle',
            'location': simulateTrip(d_id, city),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
            'origin': None,
        }

    q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        driver_mapping['status'] = q['_source']['status']
        driver_mapping['p1'] = q['_source']['p1']
        driver_mapping['destination'] = q['_source']['destination']
        driver_mapping['destinationid'] = q['_source']['destinationid']
        driver_mapping['location'] = simulateTrip(d_id, city)
        try:
            driver_mapping['p2'] = q['_source']['p2']
        except: 
            pass
    if q['found'] and (q['_source']['status'] in ['arrived']):
        t = datetime.strptime("{}".format(driver_mapping['ctime']),'%Y-%m-%dT%H:%M:%S.%fZ')
        if t < (datetime.today() - timedelta(hours = 3)):
            doc = json.dumps(driver_mapping)
            q = '{{"doc": {}}}'.format(doc)
            es.delete(index='driver', doc_type='rolling', id=last_uid)
    return(driver_mapping)

bound = loadBoundaries(boundaries_file)


In [101]:
city = random.choice(['CHI','NYC'])

driver = generateDriver(city)
u_json = json.dumps(driver).encode('utf-8')
key = json.dumps(driver['id']).encode('utf-8')
print('{}'.format(driver))




{'status': 'idle', 'origin': None, 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 5, 'name': 'driver_5', 'p2': None, 'city': 'CHI', 'p1': None, 'ctime': '2016-10-05 09:31:32.603326', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': None}


In [90]:
d_id = random.randint(1, total_drivers)
driver_mapping ={ 
            'city': city,
            'name': 'driver_{}'.format(d_id),
            'id': d_id,
            'status': 'idle',
            'location': simulateTrip(d_id, city),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
            'origin': None,
        }

   

In [92]:
q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        driver_mapping['status'] = q['_source']['status']
        driver_mapping['p1'] = q['_source']['p1']
        driver_mapping['destination'] = q['_source']['destination']
        driver_mapping['destinationid'] = q['_source']['destinationid']
        driver_mapping['location'] = simulateTrip(d_id, city)
 

In [95]:
q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])


In [100]:
[round(random.uniform(float(bnd[0]), float(bnd[2])),4),\
                round(random.uniform(float(bnd[1]), float(bnd[3])),4)]
    

[40.7169, -73.9911]

In [108]:
bnd = bound[city]
q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])
if q['found']:
        if (q['_source']['status'] in ['ontrip', 'pickup']): 
            d = q['_source']['destination']
            c = q['_source']['location']
            la = Decimal(c[0]) + (((Decimal(d[0]) - Decimal(c[0]))/step_to_dest))
            lo = Decimal(c[1]) + (((Decimal(d[1]) - Decimal(c[1]))/step_to_dest))
            print [float(str(la)), float(str(lo))]
else:
        print [round(random.uniform(float(bnd[0]), float(bnd[2])),4),\
                round(random.uniform(float(bnd[1]), float(bnd[3])),4)]

[41.9736, -87.7718]


In [136]:
city = random.choice(['CHI','NYC'])


In [137]:
city

'CHI'